# DEPRECATED

use the script generate_duplex_blacklists.py now

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import random as rd
%matplotlib inline
from collections import defaultdict
from matplotlib.patches import Rectangle
import pickle
import re
import os
import sys
sys.path.append(os.getcwd() + '/../python_scripts')
import gtools
import matplotlib.patches as mpatches
import itertools
import pandas as pd
import csv

In [ ]:
genome_file = '../data/ref/ref.fa.fai'
genome_fasta = '../data/ref/ref.fa'
sample_names = 'S3-1 S4-1 M1-1'.split()
coverage_files = [f'../data/coverage/total/justin_{sample}.bg' for sample in sample_names] # generated from bedtools genomecov -bg
# samples = itertools.chain.from_iterable([[x + '_1', x + '_2', x + '_3'] for x in 'ku80 parp2 rad5a chr8 msh2 rad7a'.split()])
# naive_files = [f'../data/variant/big_Col-0_{s}_naive.tsv' for s in range(1, 9)] + [f'../data/variant/big_{s}_naive.tsv' for s in samples]

In [ ]:
# load size of chromosomes from a fasta index
chrom_sizes = dict()
with open(genome_file, 'r') as f:
    for l in f:
        chrom_sizes[l.split()[0]] = int(l.split()[1])
# chrom_sizes

In [ ]:
# load a file with columns: chrom, pos, coverage into numpy arrays
coverage = dict()
for chrom in chrom_sizes:
    coverage[chrom] = np.zeros(chrom_sizes[chrom], dtype='i')
for file in coverage_files:
    with open(file, 'r') as f:
        for l in tqdm(f):
            fields = l.split()
            if 'e' in fields[3]: # if number is in scientific notation, python needs to convert to int using a float intermediate
                c = int(float(fields[3]))
            else:
                c = int(fields[3])
            coverage[fields[0]][int(fields[1]):int(fields[2])] += c

In [ ]:
for chrom in chrom_sizes:
    np.save(f'tmp/blacklist_coverage_sum_{chrom}.npy', coverage[chrom])

In [ ]:
coverage = dict()
for chrom in chrom_sizes:
    coverage[chrom] = np.load(f'tmp/blacklist_coverage_sum_{chrom}.npy')

In [ ]:
coverage['chr10'][100000:100010]

In [ ]:
assert np.count_nonzero(coverage['chr2'] < 0) == 0 # no negative coverages present

# Plot a histogram of read coverage

In [ ]:
# randomly sample 10M positions of the genome and get their coverage
sample_size = 10000000
chroms = rd.choices(list(chrom_sizes.keys()), k=sample_size, weights=chrom_sizes.values()) # select chroms to sample from, weighted by size
bootstrap = []
for chrom in chroms: # for each chrom selected, get the coverage at a random position
    bootstrap.append(coverage[chrom][rd.randrange(chrom_sizes[chrom])])

In [ ]:
bootstrap[:10]

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
maxcov = 1000
hist = ax.hist(bootstrap, log=False, bins=range(maxcov))
plt.show()

# Make an array of the coverage percentile by position

In [ ]:
# get the percentile value of each coverage (e.g. percentile[10] == 0.01 means 1% of positions in the genome have less than 10 coverage)
percentile = np.zeros(maxcov)
for i in tqdm(range(len(percentile))): # for each coverage amount
    lower_total = 0
    for j in range(i): # sum the occurences of lesser coverages
        lower_total += hist[0][j]
    percentile[i] = lower_total / len(bootstrap)

In [ ]:
percentile[maxcov - 1]

In [ ]:
# convert each coverage value in the genome into a percentile
cov_percentile = dict() # cov_percentile[chrom][pos] = percent of genome with lesser coverage
for chrom in tqdm(chrom_sizes):
    # apply function to coverage array to convert coverage to percentile
    cov_percentile[chrom] = np.vectorize(lambda x: percentile[x] if x < len(percentile) else percentile[-1])(coverage[chrom])
    del coverage[chrom]

In [ ]:
cov_percentile['chr1'][10000:10010]

# Make an array of duplicated regions

In [ ]:
# get the number of mismatches required for a 120bp kmer starting at a position to have a duplicate
duplicate_dist = dict() # duplicate_dist[chrom][pos] = number of bases which must be changed before the kmer has a duplicate, or 10 if no duplicate was found
for chrom in chrom_sizes:
    duplicate_dist[chrom] = np.full(chrom_sizes[chrom], 10, dtype=np.byte)

for i in range(0, -1, -1):
    with open(f'../data/ref/genmap_output_w22/120_{i}.bedgraph', 'r') as f:
        for l in tqdm(f):
            fields = l.split()
            # set values within the bedgraph interval. If occurences is 1 (unique) keep value the same, else set value to i
            duplicate_dist[fields[0]][int(fields[1]):int(fields[2])] = duplicate_dist[fields[0]][int(fields[1]):int(fields[2])] * (fields[3] == '1') + i * (fields[3]  != '1')

for i in range(4):
    print(f'{sum([np.count_nonzero(duplicate_dist[chrom] == i) for chrom in chrom_sizes]) / sum(chrom_sizes.values()) * 100}% of the genome has a (120, {i}) duplicate')

In [ ]:
duplicate_dist['Chr1']

# Make an array of polynucleotide repeat length starting/ending at each position

Most of the time, false positives generated by polynuc repeats are indels which appear at the start position of the repeat (as bowtie2 always chooses the start rather than the end if it captures the whole repeat in the read). However, mismatches can also result in rarer cases, sometimes because the repeat is near a read end or because two different repeats are adjacent. Mismatches and some indels can be placed 1bp upstream the start or 1bp downstream the end position of the repeat, so those positions are also counted. Poly C and poly G repeats don't seem to cause errors for whatever reason (less likely to slip?) and aren't counted.

e.g.
```
sequence: ACTGAAAAAAAAGTCGAGAGAGAGACTGC
array:    00088000000880554000000455000
```

In [ ]:
genome = gtools.load_genome(genome_fasta)

In [ ]:
# make zero arrays for poly 
poly_at_len = dict() # poly_at_len[chrom][pos] = length of longest poly-A or poly-T which starts or ends at the position (or the position is 1bp outside of)
poly_rep_len = dict() # poly_rep_len[chrom][pos] = length (in number of repeats) of longest di/tri-nuc repeat which starts or ends at the position (or the position is 1bp outside of)
for chrom in chrom_sizes:
    poly_at_len[chrom] = np.zeros(chrom_sizes[chrom], dtype='b')
    poly_rep_len[chrom] = np.zeros(chrom_sizes[chrom], dtype='b')

# get regex iterables for each possible nucleotide repeat
match_iters = defaultdict(lambda: [])
for chrom in chrom_sizes:
    match_iters[chrom].append((1, re.finditer('AAAA+', genome[chrom]))) # poly A, the "1" here is the length of repeat, will be "2" for di and "3" for tri
    match_iters[chrom].append((1, re.finditer('TTTT+', genome[chrom]))) # poly T
    for x in 'ACGT':
        for y in 'ACGT':
            if x != y: # two nucleotides must be different to be a dinucleotide repeat
                match_iters[chrom].append((2, re.finditer('(' + x + y + '){3,}', genome[chrom]))) # dinucleotide repeats
            for z in 'ACGT':
                if x != y or x != z or y != z: # at least one pair of nucleotides must be different to be a trinucleotide repeat
                    match_iters[chrom].append((3, re.finditer('(' + x + y + z + '){3,}', genome[chrom]))) # trinucleotide repeats
    
# iterate through each regex iterable and add the length of the repeat to the array
for chrom in match_iters:
    for replen, it in tqdm(match_iters[chrom]): # for each regex search (e.g. CT repeats)
        for match in it:
            start = match.span()[0]
            end = match.span()[1]
            poly_len = (end - start) // replen
            if replen == 1:
                poly_at_len[chrom][start - 1] = max(poly_at_len[chrom][start - 1], poly_len)
                poly_at_len[chrom][start] = max(poly_at_len[chrom][start], poly_len)
                poly_at_len[chrom][end - 1] = max(poly_at_len[chrom][end - 1], poly_len)
                if end != chrom_sizes[chrom]: # this happens if the chrom ends with a poly repeat
                    poly_at_len[chrom][end] = max(poly_at_len[chrom][end], poly_len)
            else:
                poly_rep_len[chrom][start - 1] = max(poly_rep_len[chrom][start - 1], poly_len)
                poly_rep_len[chrom][start] = max(poly_rep_len[chrom][start], poly_len)
                poly_rep_len[chrom][end - 1] = max(poly_rep_len[chrom][end - 1], poly_len)
                if end != chrom_sizes[chrom]:
                    poly_rep_len[chrom][end] = max(poly_rep_len[chrom][end], poly_len)

for i in range(4, 10):
    print(f'blacklisted {sum(np.count_nonzero(poly_at_len[chrom] == i) for chrom in chrom_sizes) / sum(chrom_sizes.values()) * 100}% for length {i} A/T nucleotide repeats')
for i in range(3, 10):
    print(f'blacklisted {sum(np.count_nonzero(poly_rep_len[chrom] == i) for chrom in chrom_sizes) / sum(chrom_sizes.values()) * 100}% for length {i} di/tri nucleotide repeats')

# Visualize blacklisted regions

In [ ]:
bin_size = 100000
fig, axs = plt.subplots(sum(['chr' in chrom for chrom in chrom_sizes]) + 1, sharex=True, sharey=True, figsize=(14, 6))
for i, chrom in tqdm(enumerate(chrom_sizes)):
    binned_cov = []
    binned_cov_per = []
    binned_dup_dist = []
    binned_at_len = []
    binned_rep_len = []
    
    # calculate the average coverage/blacklist for each bin
    for j in range(0, chrom_sizes[chrom] + bin_size, bin_size):
        # binned_cov.append(sum(coverage[chrom][j:j+bin_size]) / bin_size)
        binned_cov_per.append(np.count_nonzero((cov_percentile[chrom][j:j+bin_size] < 0.007) | (cov_percentile[chrom][j:j+bin_size] > 0.993)) / bin_size)
        binned_dup_dist.append(np.count_nonzero(duplicate_dist[chrom][j:j+bin_size] < 1) / bin_size)
        binned_at_len.append(np.count_nonzero(poly_at_len[chrom][j:j+bin_size] > 5) / bin_size)
        binned_rep_len.append(np.count_nonzero(poly_rep_len[chrom][j:j+bin_size] > 3) / bin_size)
    
    # make secondary axes to plot blacklists on
    axs[i].plot(range(len(binned_cov_per)), binned_cov_per, color='tab:orange')
    axs[i].plot(range(len(binned_dup_dist)), binned_dup_dist, color='tab:green')
    axs[i].plot(range(len(binned_at_len)), binned_at_len, color='tab:red')
    
    axs[i].set_title(chrom, y=0.35)
    
    if 'chr' not in chrom:
        break
axs[0].set_ylim(0, 1)
# manually make a legend
# cov = mpatches.Patch(color='tab:blue', label='Coverage')
dup = mpatches.Patch(color='tab:orange', label='Cov blacklist')
unmap = mpatches.Patch(color='tab:green', label='Unmap blacklist')
rep = mpatches.Patch(color='tab:red', label='Nuc repeat blacklist')
fig.legend(handles=[dup, unmap, rep], bbox_to_anchor=(1.08, 0.89))

fig.suptitle('Blacklist content of genome', y=0.96)
fig.savefig('figs/blacklist_content_of_genome.svg', dpi=300, bbox_inches='tight')

# Output blacklists

In [ ]:
for chrom in duplicate_dist:
    # np.save(f'../data/blacklist/w22/{chrom}_coverage_percentile.npy', cov_percentile[chrom])
    np.save(f'../data/blacklist/w22/{chrom}_duplicate_distance.npy', duplicate_dist[chrom])
    # np.save(f'../data/blacklist/w22/{chrom}_poly_at_length.npy', poly_at_len[chrom])
    # np.save(f'../data/blacklist/w22/{chrom}_poly_repeat_length.npy', poly_rep_len[chrom])

In [ ]:
np.count_nonzero((cov_percentile['chr1'] < 0.007)), chrom_sizes['chr1']